In [162]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [163]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Input, Concatenate, GlobalMaxPooling1D, ConvLSTM2D
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.regularizers import l1,l2,l1_l2

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [164]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
# 		print(end_ix, len(sequence))
		if end_ix == len(sequence):
			if sequence[end_ix - 1] == 17:   
# 			print(end_ix)
				seq_x, seq_y = sequence[i:end_ix], end_ix-1
			else:
				seq_x, seq_y = sequence[i:end_ix], 0                
			X.append(seq_x)
			y.append(seq_y)            
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
# 	return np.array(X), np.array(y)
	return X, y

In [165]:
with open('output.txt') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

In [166]:
processed = []
for i in range(0, len(content)): 
    a_list = content[i].split()  
    map_object = map(int, a_list)
    list_of_integers = list(map_object)
    processed.append(list_of_integers)

In [167]:
processed, len(processed)

([[0, 1, 2, 5, 8, 11, 14, 17],
  [0, 1, 2, 5, 8, 11, 14, 13, 16, 17],
  [0, 1, 2, 5, 8, 11, 14, 13, 10, 7, 4, 3, 6, 9, 12, 15, 16, 17],
  [0, 1, 2, 5, 8, 11, 14, 13, 10, 7, 6, 9, 12, 15, 16, 17],
  [0, 1, 2, 5, 8, 11, 14, 13, 10, 9, 12, 15, 16, 17],
  [0, 1, 2, 5, 8, 11, 14, 13, 12, 15, 16, 17],
  [0, 1, 2, 5, 8, 11, 10, 13, 14, 17],
  [0, 1, 2, 5, 8, 11, 10, 13, 16, 17],
  [0, 1, 2, 5, 8, 11, 10, 13, 12, 15, 16, 17],
  [0, 1, 2, 5, 8, 11, 10, 7, 4, 3, 6, 9, 12, 13, 14, 17],
  [0, 1, 2, 5, 8, 11, 10, 7, 4, 3, 6, 9, 12, 13, 16, 17],
  [0, 1, 2, 5, 8, 11, 10, 7, 4, 3, 6, 9, 12, 15, 16, 17],
  [0, 1, 2, 5, 8, 11, 10, 7, 4, 3, 6, 9, 12, 15, 16, 13, 14, 17],
  [0, 1, 2, 5, 8, 11, 10, 7, 6, 9, 12, 13, 14, 17],
  [0, 1, 2, 5, 8, 11, 10, 7, 6, 9, 12, 13, 16, 17],
  [0, 1, 2, 5, 8, 11, 10, 7, 6, 9, 12, 15, 16, 17],
  [0, 1, 2, 5, 8, 11, 10, 7, 6, 9, 12, 15, 16, 13, 14, 17],
  [0, 1, 2, 5, 8, 11, 10, 9, 12, 13, 14, 17],
  [0, 1, 2, 5, 8, 11, 10, 9, 12, 13, 16, 17],
  [0, 1, 2, 5, 8, 11, 10, 9, 1

In [168]:
# Reversing a list using reversed() 
def Reverse(lst): 
    return [ele for ele in reversed(lst)] 

processed_18 = []
for i in range(len(processed)):
    if len(processed[i]) == 18:
        processed_18.append(processed[i])
        processed_18.append(Reverse(processed[i]))
processed_18

[[0, 1, 2, 5, 8, 11, 14, 13, 10, 7, 4, 3, 6, 9, 12, 15, 16, 17],
 [17, 16, 15, 12, 9, 6, 3, 4, 7, 10, 13, 14, 11, 8, 5, 2, 1, 0],
 [0, 1, 2, 5, 8, 11, 10, 7, 4, 3, 6, 9, 12, 15, 16, 13, 14, 17],
 [17, 14, 13, 16, 15, 12, 9, 6, 3, 4, 7, 10, 11, 8, 5, 2, 1, 0],
 [0, 1, 2, 5, 8, 7, 4, 3, 6, 9, 10, 11, 14, 13, 12, 15, 16, 17],
 [17, 16, 15, 12, 13, 14, 11, 10, 9, 6, 3, 4, 7, 8, 5, 2, 1, 0],
 [0, 1, 2, 5, 8, 7, 4, 3, 6, 9, 12, 15, 16, 13, 10, 11, 14, 17],
 [17, 14, 11, 10, 13, 16, 15, 12, 9, 6, 3, 4, 7, 8, 5, 2, 1, 0],
 [0, 1, 2, 5, 4, 3, 6, 7, 8, 11, 14, 13, 10, 9, 12, 15, 16, 17],
 [17, 16, 15, 12, 9, 10, 13, 14, 11, 8, 7, 6, 3, 4, 5, 2, 1, 0],
 [0, 1, 2, 5, 4, 3, 6, 7, 8, 11, 10, 9, 12, 15, 16, 13, 14, 17],
 [17, 14, 13, 16, 15, 12, 9, 10, 11, 8, 7, 6, 3, 4, 5, 2, 1, 0],
 [0, 1, 2, 5, 4, 3, 6, 9, 10, 7, 8, 11, 14, 13, 12, 15, 16, 17],
 [17, 16, 15, 12, 13, 14, 11, 8, 7, 10, 9, 6, 3, 4, 5, 2, 1, 0],
 [0, 1, 2, 5, 4, 3, 6, 9, 12, 15, 16, 13, 10, 7, 8, 11, 14, 17],
 [17, 14, 11, 8, 7, 10, 1

In [169]:
processed_18 = processed_18[-2:]

In [170]:
# route_2 = [0, 1, 2, 5, 4, 3, 6, 7, 8, 11, 10, 9, 12, 13, 14, 17, 16, 15]
# processed_18.append(route_2)
# processed_18.append(Reverse(route_2))

In [171]:
processed_18

[[0, 3, 6, 9, 12, 15, 16, 13, 10, 7, 4, 1, 2, 5, 8, 11, 14, 17],
 [17, 14, 11, 8, 5, 2, 1, 4, 7, 10, 13, 16, 15, 12, 9, 6, 3, 0]]

In [172]:
# choose a number of time steps
n_steps = 3

samples = []
samples_output = []
# split into samples
for i in range(len(processed_18)):    
    X, y = split_sequence(processed_18[i], n_steps)
    samples.append(X)
    samples_output.append(y)

In [173]:
samples

[[[0, 3, 6],
  [3, 6, 9],
  [6, 9, 12],
  [9, 12, 15],
  [12, 15, 16],
  [15, 16, 13],
  [16, 13, 10],
  [13, 10, 7],
  [10, 7, 4],
  [7, 4, 1],
  [4, 1, 2],
  [1, 2, 5],
  [2, 5, 8],
  [5, 8, 11],
  [8, 11, 14],
  [11, 14, 17]],
 [[17, 14, 11],
  [14, 11, 8],
  [11, 8, 5],
  [8, 5, 2],
  [5, 2, 1],
  [2, 1, 4],
  [1, 4, 7],
  [4, 7, 10],
  [7, 10, 13],
  [10, 13, 16],
  [13, 16, 15],
  [16, 15, 12],
  [15, 12, 9],
  [12, 9, 6],
  [9, 6, 3],
  [6, 3, 0]]]

In [174]:
samples_output

[[9, 12, 15, 16, 13, 10, 7, 4, 1, 2, 5, 8, 11, 14, 17, 17],
 [8, 5, 2, 1, 4, 7, 10, 13, 16, 15, 12, 9, 6, 3, 0, 0]]

In [175]:
import itertools
concat = list(itertools.chain.from_iterable(samples))
concat_output = list(itertools.chain.from_iterable(samples_output))

In [176]:
concat,len(concat), concat_output,len(concat_output)

([[0, 3, 6],
  [3, 6, 9],
  [6, 9, 12],
  [9, 12, 15],
  [12, 15, 16],
  [15, 16, 13],
  [16, 13, 10],
  [13, 10, 7],
  [10, 7, 4],
  [7, 4, 1],
  [4, 1, 2],
  [1, 2, 5],
  [2, 5, 8],
  [5, 8, 11],
  [8, 11, 14],
  [11, 14, 17],
  [17, 14, 11],
  [14, 11, 8],
  [11, 8, 5],
  [8, 5, 2],
  [5, 2, 1],
  [2, 1, 4],
  [1, 4, 7],
  [4, 7, 10],
  [7, 10, 13],
  [10, 13, 16],
  [13, 16, 15],
  [16, 15, 12],
  [15, 12, 9],
  [12, 9, 6],
  [9, 6, 3],
  [6, 3, 0]],
 32,
 [9,
  12,
  15,
  16,
  13,
  10,
  7,
  4,
  1,
  2,
  5,
  8,
  11,
  14,
  17,
  17,
  8,
  5,
  2,
  1,
  4,
  7,
  10,
  13,
  16,
  15,
  12,
  9,
  6,
  3,
  0,
  0],
 32)

In [177]:
concat_output = [str(x) for x in concat_output]
concat_output,len(concat_output)

(['9',
  '12',
  '15',
  '16',
  '13',
  '10',
  '7',
  '4',
  '1',
  '2',
  '5',
  '8',
  '11',
  '14',
  '17',
  '17',
  '8',
  '5',
  '2',
  '1',
  '4',
  '7',
  '10',
  '13',
  '16',
  '15',
  '12',
  '9',
  '6',
  '3',
  '0',
  '0'],
 32)

In [178]:
b_set = set(tuple(x) for x in concat)
concat_set = [ list(x) for x in b_set ]
concat_set,len(concat_set)

([[1, 4, 7],
  [15, 12, 9],
  [2, 5, 8],
  [11, 14, 17],
  [5, 2, 1],
  [17, 14, 11],
  [3, 6, 9],
  [13, 10, 7],
  [4, 7, 10],
  [8, 11, 14],
  [12, 15, 16],
  [15, 16, 13],
  [7, 4, 1],
  [11, 8, 5],
  [14, 11, 8],
  [9, 6, 3],
  [2, 1, 4],
  [6, 9, 12],
  [16, 13, 10],
  [0, 3, 6],
  [5, 8, 11],
  [9, 12, 15],
  [4, 1, 2],
  [1, 2, 5],
  [12, 9, 6],
  [10, 13, 16],
  [13, 16, 15],
  [6, 3, 0],
  [10, 7, 4],
  [16, 15, 12],
  [8, 5, 2],
  [7, 10, 13]],
 32)

In [179]:
from collections import defaultdict

def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items())
dict_dup = dict()
for dup in sorted(list_duplicates(tuple(x) for x in concat)):
    print(dup)
    dict_dup.update({dup[0] : dup[1]})

((0, 3, 6), [0])
((1, 2, 5), [11])
((1, 4, 7), [22])
((2, 1, 4), [21])
((2, 5, 8), [12])
((3, 6, 9), [1])
((4, 1, 2), [10])
((4, 7, 10), [23])
((5, 2, 1), [20])
((5, 8, 11), [13])
((6, 3, 0), [31])
((6, 9, 12), [2])
((7, 4, 1), [9])
((7, 10, 13), [24])
((8, 5, 2), [19])
((8, 11, 14), [14])
((9, 6, 3), [30])
((9, 12, 15), [3])
((10, 7, 4), [8])
((10, 13, 16), [25])
((11, 8, 5), [18])
((11, 14, 17), [15])
((12, 9, 6), [29])
((12, 15, 16), [4])
((13, 10, 7), [7])
((13, 16, 15), [26])
((14, 11, 8), [17])
((15, 12, 9), [28])
((15, 16, 13), [5])
((16, 13, 10), [6])
((16, 15, 12), [27])
((17, 14, 11), [16])


In [180]:
concat_output_processed = []
for i in range(len(concat_set)):
    if tuple(concat_set[i]) not in dict_dup:
        concat_output_processed.append(concat_output[i])
        continue
    else:
        b = dict_dup[tuple(concat_set[i])]
        c = [concat_output[i] for i in b]
        concat_output_processed.append(set(c))

In [181]:
concat_set, len(concat_set)

([[1, 4, 7],
  [15, 12, 9],
  [2, 5, 8],
  [11, 14, 17],
  [5, 2, 1],
  [17, 14, 11],
  [3, 6, 9],
  [13, 10, 7],
  [4, 7, 10],
  [8, 11, 14],
  [12, 15, 16],
  [15, 16, 13],
  [7, 4, 1],
  [11, 8, 5],
  [14, 11, 8],
  [9, 6, 3],
  [2, 1, 4],
  [6, 9, 12],
  [16, 13, 10],
  [0, 3, 6],
  [5, 8, 11],
  [9, 12, 15],
  [4, 1, 2],
  [1, 2, 5],
  [12, 9, 6],
  [10, 13, 16],
  [13, 16, 15],
  [6, 3, 0],
  [10, 7, 4],
  [16, 15, 12],
  [8, 5, 2],
  [7, 10, 13]],
 32)

In [182]:
concat_output_processed, len(concat_output_processed)

([{'10'},
  {'6'},
  {'11'},
  {'17'},
  {'4'},
  {'8'},
  {'12'},
  {'4'},
  {'13'},
  {'17'},
  {'13'},
  {'10'},
  {'2'},
  {'2'},
  {'5'},
  {'0'},
  {'7'},
  {'15'},
  {'7'},
  {'9'},
  {'14'},
  {'16'},
  {'5'},
  {'8'},
  {'3'},
  {'15'},
  {'12'},
  {'0'},
  {'1'},
  {'9'},
  {'1'},
  {'16'}],
 32)

In [183]:
for i in range(len(concat_output_processed)):
    concat_output_processed[i] = list(concat_output_processed[i])
    while (len(concat_output_processed[i]) < 2):
        concat_output_processed[i].append(concat_output_processed[i][0])
    concat_output_processed[i] = [int(x) for x in concat_output_processed[i]]

In [184]:
concat_output_processed

[[10, 10],
 [6, 6],
 [11, 11],
 [17, 17],
 [4, 4],
 [8, 8],
 [12, 12],
 [4, 4],
 [13, 13],
 [17, 17],
 [13, 13],
 [10, 10],
 [2, 2],
 [2, 2],
 [5, 5],
 [0, 0],
 [7, 7],
 [15, 15],
 [7, 7],
 [9, 9],
 [14, 14],
 [16, 16],
 [5, 5],
 [8, 8],
 [3, 3],
 [15, 15],
 [12, 12],
 [0, 0],
 [1, 1],
 [9, 9],
 [1, 1],
 [16, 16]]

In [185]:
concat_output_processed_one = []
for i in range(len(concat_output_processed)):
    concat_output_processed_one.append(concat_output_processed[i][0])
    
concat_output_processed_two = []
for i in range(len(concat_output_processed)):
    concat_output_processed_two.append(concat_output_processed[i][1])    

In [186]:
# X, y = np.array(concat_set), np.array(concat_output_processed)
X, y1, y2 = np.array(concat_set), np.array(concat_output_processed_one), np.array(concat_output_processed_two)
X,y1,y2 

(array([[ 1,  4,  7],
        [15, 12,  9],
        [ 2,  5,  8],
        [11, 14, 17],
        [ 5,  2,  1],
        [17, 14, 11],
        [ 3,  6,  9],
        [13, 10,  7],
        [ 4,  7, 10],
        [ 8, 11, 14],
        [12, 15, 16],
        [15, 16, 13],
        [ 7,  4,  1],
        [11,  8,  5],
        [14, 11,  8],
        [ 9,  6,  3],
        [ 2,  1,  4],
        [ 6,  9, 12],
        [16, 13, 10],
        [ 0,  3,  6],
        [ 5,  8, 11],
        [ 9, 12, 15],
        [ 4,  1,  2],
        [ 1,  2,  5],
        [12,  9,  6],
        [10, 13, 16],
        [13, 16, 15],
        [ 6,  3,  0],
        [10,  7,  4],
        [16, 15, 12],
        [ 8,  5,  2],
        [ 7, 10, 13]]),
 array([10,  6, 11, 17,  4,  8, 12,  4, 13, 17, 13, 10,  2,  2,  5,  0,  7,
        15,  7,  9, 14, 16,  5,  8,  3, 15, 12,  0,  1,  9,  1, 16]),
 array([10,  6, 11, 17,  4,  8, 12,  4, 13, 17, 13, 10,  2,  2,  5,  0,  7,
        15,  7,  9, 14, 16,  5,  8,  3, 15, 12,  0,  1,  9,  1, 16]))

In [187]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.externals.joblib import dump, load
sc1 = StandardScaler()
X = sc1.fit_transform(X)
# sc2 = StandardScaler()
# y = sc2.fit_transform(y)

In [150]:
dump(sc1, 'std_scaler_nextMove.bin', compress=True)

['std_scaler_nextMove.bin']

In [188]:
X, y1,y2

(array([[-1.53493353, -0.97618706, -0.30698671],
        [ 1.33027573,  0.7592566 ,  0.1023289 ],
        [-1.33027573, -0.7592566 , -0.1023289 ],
        [ 0.51164451,  1.19311752,  1.73959133],
        [-0.71630231, -1.41004798, -1.53493353],
        [ 1.73959133,  1.19311752,  0.51164451],
        [-1.12561792, -0.54232614,  0.1023289 ],
        [ 0.92096012,  0.32539569, -0.30698671],
        [-0.92096012, -0.32539569,  0.30698671],
        [-0.1023289 ,  0.54232614,  1.12561792],
        [ 0.71630231,  1.41004798,  1.53493353],
        [ 1.33027573,  1.62697843,  0.92096012],
        [-0.30698671, -0.97618706, -1.53493353],
        [ 0.51164451, -0.10846523, -0.71630231],
        [ 1.12561792,  0.54232614, -0.1023289 ],
        [ 0.1023289 , -0.54232614, -1.12561792],
        [-1.33027573, -1.62697843, -0.92096012],
        [-0.51164451,  0.10846523,  0.71630231],
        [ 1.53493353,  0.97618706,  0.30698671],
        [-1.73959133, -1.19311752, -0.51164451],
        [-0.71630231

In [189]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [190]:
X.shape

(32, 3, 1)

In [191]:
y1.shape, y2.shape

((32,), (32,))

In [192]:
model = Sequential()
model.add(LSTM(100, return_sequences=False, kernel_regularizer=l2(0.01), input_shape=(3, 1)))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

In [193]:
model2 = Sequential()
model2.add(LSTM(100, return_sequences=False, kernel_regularizer=l2(0.01), input_shape=(3, 1)))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
model2.add(Dense(1, activation='linear'))

In [194]:
model.summary(), model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


(None, None)

In [208]:
from keras import backend
from keras import metrics 

def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

def mse(y_true, y_pred):
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

def mae(y_true, y_pred):
    return backend.mean(backend.abs(y_pred - y_true))

def mape(y_true, y_pred):
    return backend.mean(backend.abs((y_true - y_pred) / y_true)) * 100

def r2_keras(y_true, y_pred):
    SS_res =  backend.sum(backend.square(y_true - y_pred)) 
    SS_tot = backend.sum(backend.square(y_true - backend.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + backend.epsilon()) )

import os
model_filename = 'model-{epoch:03d}-{loss:03f}-{mae:03f}-{mae:03f}-simple.h5'
checkpoint_path = os.path.join('temp_seqpred/', model_filename)

model_filename2 = 'model-{epoch:03d}-{loss:03f}-simple2.h5'
checkpoint_path2 = os.path.join('temp_seqpred2/', model_filename2)

monitor = EarlyStopping(monitor = 'loss', min_delta = 1e-3, patience = 300, verbose = 1, mode = 'auto')

checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, monitor='loss',save_best_only=True, mode='auto') 
checkpoint2 = ModelCheckpoint(filepath=checkpoint_path2, verbose=1, monitor='loss',save_best_only=True, mode='auto') 


In [211]:
# Fitting our model [input_smallseq, input_medseq, input_origseq, input_smoothseq]
backend.set_epsilon(1)
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='mse', optimizer='adam', metrics=['mape', 'mse', 'mae', r2_keras,rmse])
print('Train...')
history = model.fit(X, y1, callbacks = [monitor, checkpoint], verbose = 1, batch_size = 1, nb_epoch = 100)

Train...
Epoch 1/100
32/32 [==============================] - 3s 96ms/step - loss: 0.0237 - mean_absolute_percentage_error: 1.7819 - mean_squared_error: 0.0032 - mean_absolute_error: 0.0440 - r2_keras: 0.9968 - rmse: 0.0440

Epoch 00001: loss did not improve
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0228 - mean_absolute_percentage_error: 1.5260 - mean_squared_error: 0.0024 - mean_absolute_error: 0.0380 - r2_keras: 0.9976 - rmse: 0.0380

Epoch 00002: loss did not improve
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0222 - mean_absolute_percentage_error: 1.2552 - mean_squared_error: 0.0018 - mean_absolute_error: 0.0320 - r2_keras: 0.9982 - rmse: 0.0320

Epoch 00003: loss did not improve
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0218 - mean_absolute_percentage_error: 1.0371 - mean_squared_error: 0.0014 - mean_absolute_error: 0.0272 - r2_keras: 0.9986 - rmse: 0.0272

Epoch 00004: loss did not im

32/32 [==============================] - 0s 3ms/step - loss: 0.0211 - mean_absolute_percentage_error: 0.5919 - mean_squared_error: 8.6621e-04 - mean_absolute_error: 0.0226 - r2_keras: 0.9991 - rmse: 0.0226

Epoch 00028: loss did not improve
Epoch 29/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0211 - mean_absolute_percentage_error: 0.5627 - mean_squared_error: 8.4312e-04 - mean_absolute_error: 0.0211 - r2_keras: 0.9992 - rmse: 0.0211

Epoch 00029: loss did not improve
Epoch 30/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0211 - mean_absolute_percentage_error: 0.5445 - mean_squared_error: 8.3183e-04 - mean_absolute_error: 0.0218 - r2_keras: 0.9992 - rmse: 0.0218

Epoch 00030: loss improved from 0.02108 to 0.02107, saving model to temp_seqpred/model-030-0.021066-simple.h5
Epoch 31/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0210 - mean_absolute_percentage_error: 0.5408 - mean_squared_error: 8.0838e-04 - mean_absolute_erro

32/32 [==============================] - 0s 2ms/step - loss: 0.0208 - mean_absolute_percentage_error: 0.4614 - mean_squared_error: 7.0136e-04 - mean_absolute_error: 0.0196 - r2_keras: 0.9993 - rmse: 0.0196

Epoch 00055: loss improved from 0.02085 to 0.02083, saving model to temp_seqpred/model-055-0.020830-simple.h5
Epoch 56/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0208 - mean_absolute_percentage_error: 0.4290 - mean_squared_error: 6.8701e-04 - mean_absolute_error: 0.0191 - r2_keras: 0.9993 - rmse: 0.0191

Epoch 00056: loss improved from 0.02083 to 0.02081, saving model to temp_seqpred/model-056-0.020811-simple.h5
Epoch 57/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0208 - mean_absolute_percentage_error: 0.4475 - mean_squared_error: 7.1810e-04 - mean_absolute_error: 0.0195 - r2_keras: 0.9993 - rmse: 0.0195

Epoch 00057: loss did not improve
Epoch 58/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0208 - mean_absolute_pe

32/32 [==============================] - 0s 3ms/step - loss: 0.0206 - mean_absolute_percentage_error: 0.3801 - mean_squared_error: 6.2578e-04 - mean_absolute_error: 0.0176 - r2_keras: 0.9994 - rmse: 0.0176

Epoch 00082: loss improved from 0.02065 to 0.02064, saving model to temp_seqpred/model-082-0.020644-simple.h5
Epoch 83/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0206 - mean_absolute_percentage_error: 0.3899 - mean_squared_error: 6.3948e-04 - mean_absolute_error: 0.0180 - r2_keras: 0.9994 - rmse: 0.0180

Epoch 00083: loss did not improve
Epoch 84/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0207 - mean_absolute_percentage_error: 0.3943 - mean_squared_error: 6.5855e-04 - mean_absolute_error: 0.0183 - r2_keras: 0.9993 - rmse: 0.0183

Epoch 00084: loss did not improve
Epoch 85/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0207 - mean_absolute_percentage_error: 0.3885 - mean_squared_error: 6.6954e-04 - mean_absolute_erro

In [42]:
# Fitting our model [input_smallseq, input_medseq, input_origseq, input_smoothseq]
adam = optimizers.Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model2.compile(loss='mse', optimizer='adam', metrics=[rmse])
print('Train model 2...')
history = model2.fit(X, y2, callbacks = [monitor, checkpoint2], verbose = 1, batch_size = 1, nb_epoch = 500)

Train model 2...
Epoch 1/500
62/62 [==============================] - 2s 25ms/step - loss: 0.6392 - rmse: 0.4746

Epoch 00001: rmse did not improve
Epoch 2/500
62/62 [==============================] - 0s 5ms/step - loss: 0.6806 - rmse: 0.4549

Epoch 00002: rmse did not improve
Epoch 3/500
62/62 [==============================] - 0s 4ms/step - loss: 0.5496 - rmse: 0.4274

Epoch 00003: rmse did not improve
Epoch 4/500
62/62 [==============================] - 0s 4ms/step - loss: 0.4951 - rmse: 0.4115

Epoch 00004: rmse did not improve
Epoch 5/500
62/62 [==============================] - 0s 4ms/step - loss: 0.6938 - rmse: 0.4406

Epoch 00005: rmse did not improve
Epoch 6/500
62/62 [==============================] - 0s 5ms/step - loss: 1.3539 - rmse: 0.6574

Epoch 00006: rmse did not improve
Epoch 7/500
62/62 [==============================] - 0s 4ms/step - loss: 3.2234 - rmse: 0.7977

Epoch 00007: rmse did not improve
Epoch 8/500
62/62 [==============================] - 0s 4ms/step - loss:

62/62 [==============================] - 0s 4ms/step - loss: 0.3650 - rmse: 0.3037

Epoch 00062: rmse did not improve
Epoch 63/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3347 - rmse: 0.2755

Epoch 00063: rmse did not improve
Epoch 64/500
62/62 [==============================] - 0s 4ms/step - loss: 0.7205 - rmse: 0.4131

Epoch 00064: rmse did not improve
Epoch 65/500
62/62 [==============================] - 0s 4ms/step - loss: 0.5850 - rmse: 0.4474

Epoch 00065: rmse did not improve
Epoch 66/500
62/62 [==============================] - 0s 4ms/step - loss: 0.4325 - rmse: 0.3543

Epoch 00066: rmse did not improve
Epoch 67/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3977 - rmse: 0.3468

Epoch 00067: rmse did not improve
Epoch 68/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3864 - rmse: 0.3404

Epoch 00068: rmse did not improve
Epoch 69/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3534 - rmse: 0.2761


62/62 [==============================] - 0s 4ms/step - loss: 0.7340 - rmse: 0.3998

Epoch 00123: rmse did not improve
Epoch 124/500
62/62 [==============================] - 0s 5ms/step - loss: 1.9049 - rmse: 0.5641

Epoch 00124: rmse did not improve
Epoch 125/500
62/62 [==============================] - 0s 4ms/step - loss: 0.6151 - rmse: 0.4263

Epoch 00125: rmse did not improve
Epoch 126/500
62/62 [==============================] - 0s 4ms/step - loss: 0.4181 - rmse: 0.3105

Epoch 00126: rmse did not improve
Epoch 127/500
62/62 [==============================] - 0s 4ms/step - loss: 0.4137 - rmse: 0.2417

Epoch 00127: rmse did not improve
Epoch 128/500
62/62 [==============================] - 0s 5ms/step - loss: 0.3934 - rmse: 0.2576

Epoch 00128: rmse did not improve
Epoch 129/500
62/62 [==============================] - 0s 5ms/step - loss: 0.3835 - rmse: 0.2328

Epoch 00129: rmse did not improve
Epoch 130/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3497 - rmse: 

62/62 [==============================] - 0s 4ms/step - loss: 0.3328 - rmse: 0.1718

Epoch 00184: rmse improved from 0.17403 to 0.17179, saving model to temp_seqpred2/model-184-0.332754-0.171795-simple2.h5
Epoch 185/500
62/62 [==============================] - 0s 5ms/step - loss: 0.3067 - rmse: 0.1805

Epoch 00185: rmse did not improve
Epoch 186/500
62/62 [==============================] - 0s 5ms/step - loss: 0.3278 - rmse: 0.1779

Epoch 00186: rmse did not improve
Epoch 187/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3159 - rmse: 0.1889

Epoch 00187: rmse did not improve
Epoch 188/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3418 - rmse: 0.1822

Epoch 00188: rmse did not improve
Epoch 189/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3102 - rmse: 0.1734

Epoch 00189: rmse did not improve
Epoch 190/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3185 - rmse: 0.1755

Epoch 00190: rmse did not improve
Epo

62/62 [==============================] - 0s 5ms/step - loss: 0.4538 - rmse: 0.3751

Epoch 00245: rmse did not improve
Epoch 246/500
62/62 [==============================] - 0s 5ms/step - loss: 0.3607 - rmse: 0.2653

Epoch 00246: rmse did not improve
Epoch 247/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3378 - rmse: 0.2603

Epoch 00247: rmse did not improve
Epoch 248/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3094 - rmse: 0.2361

Epoch 00248: rmse did not improve
Epoch 249/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3357 - rmse: 0.2513

Epoch 00249: rmse did not improve
Epoch 250/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3223 - rmse: 0.2480

Epoch 00250: rmse did not improve
Epoch 251/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3837 - rmse: 0.2721

Epoch 00251: rmse did not improve
Epoch 252/500
62/62 [==============================] - 0s 4ms/step - loss: 3.2688 - rmse: 

62/62 [==============================] - 0s 4ms/step - loss: 1.4499 - rmse: 0.6694

Epoch 00307: rmse did not improve
Epoch 308/500
62/62 [==============================] - 0s 4ms/step - loss: 3.1442 - rmse: 0.6861

Epoch 00308: rmse did not improve
Epoch 309/500
62/62 [==============================] - 0s 5ms/step - loss: 3.2823 - rmse: 1.0306

Epoch 00309: rmse did not improve
Epoch 310/500
62/62 [==============================] - 0s 5ms/step - loss: 2.4120 - rmse: 0.8154

Epoch 00310: rmse did not improve
Epoch 311/500
62/62 [==============================] - 0s 4ms/step - loss: 2.1528 - rmse: 0.6376

Epoch 00311: rmse did not improve
Epoch 312/500
62/62 [==============================] - 0s 4ms/step - loss: 2.2265 - rmse: 0.6716

Epoch 00312: rmse did not improve
Epoch 313/500
62/62 [==============================] - 0s 4ms/step - loss: 1.8975 - rmse: 0.6128

Epoch 00313: rmse did not improve
Epoch 314/500
62/62 [==============================] - 0s 4ms/step - loss: 1.8215 - rmse: 

62/62 [==============================] - 0s 4ms/step - loss: 0.4014 - rmse: 0.3998

Epoch 00369: rmse did not improve
Epoch 370/500
62/62 [==============================] - 0s 4ms/step - loss: 0.3154 - rmse: 0.2872

Epoch 00370: rmse did not improve
Epoch 371/500
62/62 [==============================] - 0s 4ms/step - loss: 0.2708 - rmse: 0.2642

Epoch 00371: rmse did not improve
Epoch 372/500
62/62 [==============================] - 0s 5ms/step - loss: 0.3037 - rmse: 0.2686

Epoch 00372: rmse did not improve
Epoch 373/500
62/62 [==============================] - 0s 5ms/step - loss: 0.2451 - rmse: 0.2145

Epoch 00373: rmse did not improve
Epoch 374/500
62/62 [==============================] - 0s 4ms/step - loss: 0.2820 - rmse: 0.2264

Epoch 00374: rmse did not improve
Epoch 375/500
62/62 [==============================] - 0s 4ms/step - loss: 0.2457 - rmse: 0.2070

Epoch 00375: rmse did not improve
Epoch 376/500
62/62 [==============================] - 0s 5ms/step - loss: 0.2972 - rmse: 

62/62 [==============================] - 0s 4ms/step - loss: 0.2453 - rmse: 0.1694

Epoch 00429: rmse did not improve
Epoch 430/500
62/62 [==============================] - 0s 5ms/step - loss: 0.2140 - rmse: 0.1432

Epoch 00430: rmse did not improve
Epoch 431/500
62/62 [==============================] - 0s 4ms/step - loss: 0.2377 - rmse: 0.1922

Epoch 00431: rmse did not improve
Epoch 432/500
62/62 [==============================] - 0s 5ms/step - loss: 0.2054 - rmse: 0.1470

Epoch 00432: rmse did not improve
Epoch 433/500
62/62 [==============================] - 0s 6ms/step - loss: 0.1994 - rmse: 0.1441

Epoch 00433: rmse did not improve
Epoch 434/500
62/62 [==============================] - 0s 4ms/step - loss: 0.1999 - rmse: 0.1383

Epoch 00434: rmse did not improve
Epoch 435/500
62/62 [==============================] - 0s 4ms/step - loss: 0.2416 - rmse: 0.2032

Epoch 00435: rmse did not improve
Epoch 436/500
62/62 [==============================] - 0s 4ms/step - loss: 0.2458 - rmse: 

62/62 [==============================] - 0s 4ms/step - loss: 1.8154 - rmse: 0.6265

Epoch 00491: rmse did not improve
Epoch 492/500
62/62 [==============================] - 0s 4ms/step - loss: 1.6925 - rmse: 0.5450

Epoch 00492: rmse did not improve
Epoch 493/500
62/62 [==============================] - 0s 4ms/step - loss: 1.7357 - rmse: 0.5925

Epoch 00493: rmse did not improve
Epoch 494/500
62/62 [==============================] - 0s 5ms/step - loss: 1.6715 - rmse: 0.5491

Epoch 00494: rmse did not improve
Epoch 495/500
62/62 [==============================] - 0s 4ms/step - loss: 1.7099 - rmse: 0.6470

Epoch 00495: rmse did not improve
Epoch 496/500
62/62 [==============================] - 0s 4ms/step - loss: 1.3604 - rmse: 0.4862

Epoch 00496: rmse did not improve
Epoch 497/500
62/62 [==============================] - 0s 4ms/step - loss: 1.5777 - rmse: 0.4917

Epoch 00497: rmse did not improve
Epoch 498/500
62/62 [==============================] - 0s 4ms/step - loss: 1.5542 - rmse: 

In [93]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_models/lstm_simplemodels.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
# model.save_weights("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_weights/MSLSTM_18pts_weight.h5")
print("Saved model to disk")

Saved model to disk


In [84]:
from keras.models import model_from_json
# load json and create model
# json_file = open("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_models/MSLSTM_18pts_model.json", 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_weights/model-013-0.231717-0.140464-simple_.h5")
model2.load_weights("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_weights/model-237-0.067243-0.042658-simple2.h5")
print("Loaded model from disk")

Loaded model from disk


In [199]:
# test_input = np.array([[1,  4,  7],
#                      [0, 1, 2],
#                       [ 3,  6,  9]]) #[10, 8], [5, 18]
test_input = np.array([[ 1,  4,  7],
        [15, 12,  9],
        [ 2,  5,  8],
        [11, 14, 17],
        [ 5,  2,  1],
        [17, 14, 11],
        [ 3,  6,  9],
        [13, 10,  7],
        [ 4,  7, 10],
        [ 8, 11, 14]
                      ]) #[10,10], [5,5], [9,9] [12,12] 5, ? [10,6], [0,4]
        
test_input = sc1.transform(test_input)
test_input = test_input.reshape((10, 3, 1))
test_output = model.predict(test_input, verbose=0)
test_output2 = model2.predict(test_input, verbose=0)
# test_output = sc2.inverse_transform(test_output)
print(np.round(test_output))
print(np.round(test_output2))

[[10.]
 [ 6.]
 [11.]
 [16.]
 [ 4.]
 [ 8.]
 [12.]
 [ 4.]
 [13.]
 [17.]]
[[-0.]
 [ 0.]
 [-0.]
 [ 0.]
 [-0.]
 [ 0.]
 [-0.]
 [ 0.]
 [-0.]
 [ 0.]]
